In [1]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import numpy as np
import time
import os
from keras.preprocessing.image import ImageDataGenerator

D:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def load_data(path='./fer2013/fer2013.csv'):
    a=time.time()
    Y = []
    X = []
    first = True
    for line in open(path):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])
    X, Y = np.array(X), np.array(Y)
    b=time.time()
    print("cost-time is"+str(b-a)+"s")
    Y[Y>0]=Y[Y>0]-1
    Y = to_categorical(Y)
    X=X.reshape([-1,48,48,1])
    X = X / 255.
    train_x=X[0:33000]
    train_y=Y[0:33000]
    test_x=X[33000:35887]
    test_y=Y[33000:35887]
    return train_x,train_y,test_x,test_y

In [3]:
x_train, y_train, x_test, y_test= load_data()
print(x_train.shape)

cost-time is24.07335591316223s
(33000, 48, 48, 1)


In [4]:
#ResNet Block
def resnet_block(inputs,num_filters=16,
                  kernel_size=3,strides=1,
                  activation='relu'):
    x = Conv2D(num_filters,kernel_size=kernel_size,strides=strides,padding='same',
           kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(inputs)
    x = BatchNormalization()(x)
    if(activation):
        x = Activation('relu')(x)
    return x

In [5]:
# 建一个20层的ResNet网络 
def resnet_v1(input_shape):
    inputs = Input(shape=input_shape)# Input层，用来当做占位使用

    #第一层
    x = resnet_block(inputs)
    print('layer1,xshape:',x.shape)
    # 第2~7层
    for i in range(6):
        a = resnet_block(inputs = x)
        b = resnet_block(inputs=a,activation=None)
        x = keras.layers.add([x,b])
        x = Activation('relu')(x)
    # out：32*32*16
    # 第8~13层
    for i in range(6):
        if i == 0:
            a = resnet_block(inputs = x,strides=2,num_filters=32)
        else:
            a = resnet_block(inputs = x,num_filters=32)
        b = resnet_block(inputs=a,activation=None,num_filters=32)
        if i==0:
            x = Conv2D(32,kernel_size=3,strides=2,padding='same',
                       kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(x)
        x = keras.layers.add([x,b])
        x = Activation('relu')(x)
    # out:16*16*32
    # 第14~19层
    for i in range(6):
        if i ==0 :
            a = resnet_block(inputs = x,strides=2,num_filters=64)
        else:
            a = resnet_block(inputs = x,num_filters=64)

        b = resnet_block(inputs=a,activation=None,num_filters=64)
        if i == 0:
            x = Conv2D(64,kernel_size=3,strides=2,padding='same',
                       kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(x)
        x = keras.layers.add([x,b])# 相加操作，要求x、b shape完全一致
        x = Activation('relu')(x)
    # out:8*8*64
    # 第20层   
    x = AveragePooling2D(pool_size=2)(x)
    # out:4*4*64
    y = Flatten()(x)
    # out:1024
    outputs = Dense(6,activation='softmax',
                    kernel_initializer='he_normal')(y)

    #初始化模型
    #之前的操作只是将多个神经网络层进行了相连，通过下面这一句的初始化操作，才算真正完成了一个模型的结构初始化
    model = Model(inputs=inputs,outputs=outputs)
    return model

In [6]:
model = resnet_v1((48,48,1))

model.compile(loss='categorical_crossentropy',
optimizer=Adam(),
metrics=['accuracy'])

#model.summary()

layer1,xshape: (?, 48, 48, 16)


In [7]:
checkpoint = ModelCheckpoint(filepath='./resnet_ckpt.h5',monitor='val_acc',
                             verbose=1,save_best_only=True)
def lr_sch(epoch):
    #200 total
    if epoch <50:
        return 1e-3
    if 50<=epoch<100:
        return 1e-4
    if epoch>=100:
        return 1e-5
lr_scheduler = LearningRateScheduler(lr_sch)
lr_reducer = ReduceLROnPlateau(monitor='val_acc',factor=0.2,patience=5,
                               mode='max',min_lr=1e-3)
callbacks = [checkpoint,lr_scheduler,lr_reducer]

In [15]:
x=np.load('x_train.npy')
y=np.load('y_train.npy')

In [16]:
print(x.shape)

(100000, 48, 48, 1)


In [17]:
model.fit(x,y,batch_size=128,epochs=2,validation_data=(x_test,y_test),verbose=1,callbacks=callbacks)

Train on 100000 samples, validate on 2887 samples
Epoch 1/2
100000/100000 [==============================] - 6605s 66ms/step - loss: 3.8246 - acc: 0.3798 - val_loss: 1.6154 - val_acc: 0.4659

Epoch 00001: val_acc improved from 0.22826 to 0.46588, saving model to ./resnet_ckpt.h5
Epoch 2/2
100000/100000 [==============================] - 6615s 66ms/step - loss: 1.4533 - acc: 0.5173 - val_loss: 1.3829 - val_acc: 0.5438

Epoch 00002: val_acc improved from 0.46588 to 0.54382, saving model to ./resnet_ckpt.h5


In [20]:
model.fit(x,y,batch_size=128,epochs=20,validation_data=(x_test,y_test),verbose=1,callbacks=callbacks)

Train on 100000 samples, validate on 2887 samples
Epoch 1/20
100000/100000 [==============================] - 6805s 68ms/step - loss: 1.1498 - acc: 0.6298 - val_loss: 1.2510 - val_acc: 0.5951

Epoch 00001: val_acc improved from 0.54555 to 0.59508, saving model to ./resnet_ckpt.h5
Epoch 2/20
100000/100000 [==============================] - 6684s 67ms/step - loss: 1.0864 - acc: 0.6552 - val_loss: 1.3023 - val_acc: 0.5691

Epoch 00002: val_acc did not improve
Epoch 3/20
100000/100000 [==============================] - 6709s 67ms/step - loss: 1.0283 - acc: 0.6759 - val_loss: 1.2301 - val_acc: 0.6058

Epoch 00003: val_acc improved from 0.59508 to 0.60582, saving model to ./resnet_ckpt.h5
Epoch 4/20
100000/100000 [==============================] - 6607s 66ms/step - loss: 0.9864 - acc: 0.6955 - val_loss: 1.2009 - val_acc: 0.6273

Epoch 00004: val_acc improved from 0.60582 to 0.62729, saving model to ./resnet_ckpt.h5
Epoch 5/20
100000/100000 [==============================] - 6606s 66ms/step -

In [21]:
datagen = ImageDataGenerator(
    rotation_range = 10,
    width_shift_range=0.03,
    height_shift_range=0.03,
    zoom_range = 0.05,
    horizontal_flip = True,
)

In [ ]:
epochs=10
for e in range(epochs):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=1024):
        model.fit(x_batch, y_batch,batch_size=128,validation_data=(x_test,y_test),verbose=1,callbacks=callbacks)
        batches += 1
        if batches >= len(x_train) / 1024:
            # 我们需要手动打破循环，
            # 因为生成器会无限循环
            break

Epoch 0
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9677 - acc: 0.7441 - val_loss: 1.4881 - val_acc: 0.6297

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9560 - acc: 0.7686 - val_loss: 1.5674 - val_acc: 0.6024

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9528 - acc: 0.7568 - val_loss: 1.5889 - val_acc: 0.5940

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 1.0326 - acc: 0.7412 - val_loss: 1.5248 - val_acc: 0.6176

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================]


Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9903 - acc: 0.7412 - val_loss: 1.5063 - val_acc: 0.6117

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 1.0103 - acc: 0.7324 - val_loss: 1.5266 - val_acc: 0.6117

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9810 - acc: 0.7578 - val_loss: 1.5165 - val_acc: 0.6107

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 1.0192 - acc: 0.7490 - val_loss: 1.4737 - val_acc: 0.6259

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [=

1024/1024 [==============================] - 130s 127ms/step - loss: 1.0053 - acc: 0.7373 - val_loss: 1.3847 - val_acc: 0.6252

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 1.0108 - acc: 0.7363 - val_loss: 1.3701 - val_acc: 0.6249

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 131s 128ms/step - loss: 1.0120 - acc: 0.7314 - val_loss: 1.3652 - val_acc: 0.6356

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9790 - acc: 0.7568 - val_loss: 1.4094 - val_acc: 0.6145

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 129s 126ms/step - loss: 1.0112 - acc: 0.7441 - val_loss: 1.4686

Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 1.0229 - acc: 0.7207 - val_loss: 1.3762 - val_acc: 0.6335

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9477 - acc: 0.7607 - val_loss: 1.3931 - val_acc: 0.6249

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 1.0299 - acc: 0.7158 - val_loss: 1.4088 - val_acc: 0.6245

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 1.0162 - acc: 0.7285 - val_loss: 1.3822 - val_acc: 0.6360

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 


Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 1.0065 - acc: 0.7383 - val_loss: 1.3684 - val_acc: 0.6418

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9513 - acc: 0.7490 - val_loss: 1.3752 - val_acc: 0.6380

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9327 - acc: 0.7676 - val_loss: 1.3692 - val_acc: 0.6377

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9510 - acc: 0.7529 - val_loss: 1.5001 - val_acc: 0.6079

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [=

1024/1024 [==============================] - 130s 127ms/step - loss: 0.9123 - acc: 0.7646 - val_loss: 1.4149 - val_acc: 0.6249

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9788 - acc: 0.7441 - val_loss: 1.4758 - val_acc: 0.6263

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9686 - acc: 0.7373 - val_loss: 1.3719 - val_acc: 0.6474

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 1.0281 - acc: 0.7314 - val_loss: 1.3746 - val_acc: 0.6411

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9967 - acc: 0.7402 - val_loss: 1.3893

Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9795 - acc: 0.7412 - val_loss: 1.3875 - val_acc: 0.6332

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9405 - acc: 0.7715 - val_loss: 1.4389 - val_acc: 0.6297

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 1.0909 - acc: 0.7129 - val_loss: 1.4515 - val_acc: 0.6214

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9726 - acc: 0.7568 - val_loss: 1.3847 - val_acc: 0.6373

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 


Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9272 - acc: 0.7451 - val_loss: 1.3439 - val_acc: 0.6373

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 132s 129ms/step - loss: 0.9998 - acc: 0.7432 - val_loss: 1.3754 - val_acc: 0.6315

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9592 - acc: 0.7529 - val_loss: 1.3690 - val_acc: 0.6276

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9643 - acc: 0.7637 - val_loss: 1.3416 - val_acc: 0.6474

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [=

1024/1024 [==============================] - 130s 127ms/step - loss: 1.0085 - acc: 0.7148 - val_loss: 1.4055 - val_acc: 0.6214

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9796 - acc: 0.7402 - val_loss: 1.4043 - val_acc: 0.6166

Epoch 00001: val_acc did not improve
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.9811 - acc: 0.7520 - val_loss: 1.3998 - val_acc: 0.6211

Epoch 00001: val_acc did not improve
Train on 232 samples, validate on 2887 samples
Epoch 1/1
232/232 [==============================] - 78s 334ms/step - loss: 0.9035 - acc: 0.7672 - val_loss: 1.3878 - val_acc: 0.6263

Epoch 00001: val_acc did not improve
Epoch 9
Train on 1024 samples, validate on 2887 samples
Epoch 1/1
1024/1024 [==============================] - 130s 127ms/step - loss: 0.8935 - acc: 0.7822 - val_loss: 1.


Epoch 00001: val_acc did not improve


In [ ]:
epochs=100
for e in range(epochs):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=1024):
        model.fit(x_batch, y_batch,batch_size=256,validation_data=(x_test,y_test),verbose=1,callbacks=callbacks)
        batches += 1
        if batches >= len(x_train) / 1024:
            # 我们需要手动打破循环，
            # 因为生成器会无限循环
            break

In [ ]:
model.fit_generator(datagen.flow(x_train,y_train,batch_size=256),steps_per_epoch = 256,epochs = 150,validation_data=(x_test,y_test),workers=4,verbose=1,callbacks=callbacks)